In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm

# Efecto del Tracking Escolar y Inferencia con Muestras Agrupadas

En contextos donde las unidades de observación están naturalmente agrupadas (como estudiantes
dentro de escuelas), la independencia entre observaciones puede no cumplirse. Esto afecta la inferencia
estadística, ya que los errores estándar convencionales pueden subestimar la varianza real
del estimador. En este ejercicio trabajaremos con datos de un experimento aleatorizado en escuelas
de Kenia (Duflo, Dupas y Kremer, 2011), que evaluó el efecto de dividir a los estudiantes en clases
según su nivel de rendimiento inicial (“tracking”).
### Objetivo
Replicar la estimación del efecto promedio del tracking sobre el rendimiento escolar con inferencia
adecuada para datos agrupados.
### Base de datos
Utilice el archivo DDK2011.dta. La variable dependiente es el puntaje estandarizado TestScore, y
la variable de tratamiento es Tracking, que indica si la escuela aplicó tracking.

In [15]:
url=r"C:\Users\Camil\Documents\GitHub\ECOP2037_CE\hmw2\DDK2011.dta"
df = pd.read_stata(url)

## Parte I: Estimación básica
1. Estime el siguiente modelo por Mínimos Cuadrados Ordinarios (OLS):
$$
\text{TestScore}_{ig} = \alpha + \gamma \text{Tracking}_g + e_{ig}
$$

2. Reporte el valor de $\hat{\gamma}$ y su error estándar usando:
(a) Errores estándar convencionales (homocedásticos).

In [3]:
y=df['totalscore']
x=df['tracking']
x = sm.add_constant(x)  
model_conventional = sm.OLS(y, x).fit()
hat_gamma=model_conventional.params[1]
hat_se=model_conventional.bse[1]
print(f'Gamma estimado=', hat_gamma)
print(f'Error estandar estimado=', hat_se)

Gamma estimado= 1.2581967566352972
Error estandar estimado= 0.2389277353144617


C:\Users\Camil\AppData\Local\Temp\ipykernel_20108\3137233708.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  hat_gamma=model_conventional.params[1]
C:\Users\Camil\AppData\Local\Temp\ipykernel_20108\3137233708.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  hat_se=model_conventional.bse[1]


(b) Errores estándar robustos agrupados por escuela.

In [4]:
model_clustered = sm.OLS(y, x).fit( cov_type='cluster', cov_kwds={'groups': df['schoolid']})
hat_gamma1=model_clustered.params[1]
hat_se1=model_clustered.bse[1]
print(f'Gamma estimado=', hat_gamma1)
print(f'Error estandar estimado=', hat_se1)

Gamma estimado= 1.2581967566352972
Error estandar estimado= 0.7037256469657598


C:\Users\Camil\AppData\Local\Temp\ipykernel_20108\812076604.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  hat_gamma1=model_clustered.params[1]
C:\Users\Camil\AppData\Local\Temp\ipykernel_20108\812076604.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  hat_se1=model_clustered.bse[1]


3. Comente las diferencias. ¿Qué implicancias tiene para la significancia estadística del efecto estimado?

El error estándar convencional es 0.24, mientras que el error estándar clustered por escuela es 0.70. Esto muestra que al no tener en cuenta la agrupación por escuela,es decir, que los estudiantes de una misma escuela pueden estar correlacionados, el error estándar se subestima. Como resultado, el efecto de Tracking parece ser estadísticamente significativo con errores convencionales, pero pierde significancia cuando se usan errores agrupados. 

## Parte II: Desafíos adicionales
### Challenge 1: Robustez con controles individuales
Agregue las siguientes variables de control al modelo anterior:
- Edad del estudiante.
- Género.


¿Cambia la magnitud o significancia de $\hat{\gamma}$? ¿Por qué?

In [ ]:
model_clustered2 = smf.ols('totalscore ~ tracking + agetest + girl ', data=df).fit()
hat_gamma2=model_clustered2.params[1]
hat_se2=model_clustered2.bse[1]
print('El estimador 𝛾 es :',hat_gamma2)

El estimador 𝛾 es : 1.193529925607351


C:\Users\Camil\AppData\Local\Temp\ipykernel_20108\2087989952.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  hat_gamma2=model_clustered2.params[1]
C:\Users\Camil\AppData\Local\Temp\ipykernel_20108\2087989952.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  hat_se2=model_clustered2.bse[1]


El estimador que pasa de 1.25 a 1.19 al incluir controles quiere decir que una pequeña parte del efecto que inicialmente atribuías al tracking en realidad estaba relacionada con diferencias en género o edad de los estudiantes. Al incluir estos controles, este sesgo se esta corrigiendo.

## Challenge 2: Heterogeneidad del efecto del tracking
Cree una variable BottomHalf que sea 1 si el estudiante estaba en la mitad inferior del puntaje
inicial en su escuela. Estime:
$$
\text{TestScore}_{ig} = \alpha + \gamma_1  \text{Tracking}_g + \gamma_2 \text{BottomHalf}_ig + \gamma_3 \text{Tracking}_g * \text{BottomHalf}_ig + e_{ig}
$$


Interprete el coeficiente $\hat{\gamma_3}$
¿Es diferente el efecto del tracking para estudiantes de menor rendimiento
inicial?

In [26]:
## crear variable bottomhalf
df['tracking_bottomhalf']=df['tracking']*df['bottomhalf']
model_clustered3 = smf.ols('totalscore ~ tracking + bottomhalf + tracking_bottomhalf', data=df).fit()
hat_gamma3=model_clustered3.params[3]
hat_se3=model_clustered3.bse[1]
print("El estimador 𝛾_3 es:",hat_gamma3)

El estimador 𝛾_3 es: -0.336170567137686


C:\Users\Camil\AppData\Local\Temp\ipykernel_20108\4250876937.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  hat_gamma3=model_clustered3.params[3]
C:\Users\Camil\AppData\Local\Temp\ipykernel_20108\4250876937.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  hat_se3=model_clustered3.bse[1]


$𝛾_3$ muestra cuánto menor es el efecto del tracking para los estudiantes de la mitad inferior en comparación con los de la mitad superior. Esto significa que el efecto del tracking sobre los estudiantes de la mitad inferior es 0.3362 puntos menor que el efecto sobre los estudiantes de la mitad superior.

## Challenge 3: Inferencia errónea por no agrupar
Compare los errores estándar de $\hat{\gamma}$ usando:
- Errores convencionales.
- Errores robustos de White (HC0)
- Errores robustos agrupados por escuela.


¿En cuál caso cambia la significancia estadística del efecto de tracking?

In [38]:
print("El error convencionales es:",hat_se3)

##errores robustos de white
model_white = smf.ols('totalscore ~ tracking + bottomhalf + tracking_bottomhalf', data=df).fit(cov_type='HC0')
hat_se4=model_white.bse[1]
print("El error robusto de white es:",hat_se4)

##errores clusteres
# debemos eliminar na porque sino sale error 
vars_needed = ['totalscore', 'tracking', 'bottomhalf', 'schoolid']
df_reg = df[vars_needed].dropna().copy()
df_reg['tracking_bottomhalf'] = df_reg['tracking'] * df_reg['bottomhalf']


model_clustered = smf.ols('totalscore ~ tracking + bottomhalf + tracking_bottomhalf', data=df_reg).fit( cov_type='cluster', cov_kwds={'groups': df_reg['schoolid']})
hat_se5=model_clustered.bse[1]
print("El error robusto agrupado es:",hat_se5)


El error convencionales es: 0.3204300840465834
El error robusto de white es: 0.3440845115981084
El error robusto agrupado es: 0.8534855693048853


C:\Users\Camil\AppData\Local\Temp\ipykernel_20108\3396091151.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  hat_se4=model_white.bse[1]
C:\Users\Camil\AppData\Local\Temp\ipykernel_20108\3396091151.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  hat_se5=model_clustered.bse[1]


Como podemos observar, los valores de los errores estandar son muy similares para los valores que utilizan errores estandar convencionales y HCO. Viendo estos errores, podriamos decir que el efecto de Tracking en totalscore es significativo. Sin embargo, al estimar el mismo modelo usando errores estandar clustered al nivel de escuela observamos que este efecto de hecho no es significativo. Esto demuestra que no usar errores estandar clustered, nos puede llevar a falsos positivos.